In [1]:
import polars as pl
from polars.exceptions import ColumnNotFoundError
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['timezone'] = 'Europe/Moscow'

from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
import pickle
import json
from tqdm.notebook import tqdm
from itertools import product
import random

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score

from bot.core.db.postgres_manager import DBManager
from bot.config.credentials import host, user, password, db_name
db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)

from bot.analysis.backtesting import backtest
from bot.utils.pair_trading import *
from bot.analysis.strategy_analysis import analyze_strategy
from bot.utils.files import load_config

with open("./data/coin_information.pkl", "rb") as f:
     coin_information = pickle.load(f)

#### Создание датасета

In [2]:
def run_test(df, wind, in_, out_, open_method, dist_in, dist_out, force_close):
    cols = ['time', 'ts', token_1, f'{token_1}_size', f'{token_1}_bid_price', f'{token_1}_ask_price',
            f'{token_1}_bid_size', f'{token_1}_ask_size', token_2, f'{token_2}_size',
            f'{token_2}_bid_price', f'{token_2}_ask_price', f'{token_2}_bid_size', f'{token_2}_ask_size',
            f'spread_{wind}_{tf}', f'spread_mean_{wind}_{tf}', f'spread_std_{wind}_{tf}', f'z_score_{wind}_{tf}']

    tdf = df.select(cols).rename({f'z_score_{wind}_{tf}': 'z_score', f'spread_{wind}_{tf}': 'spread',
                f'spread_mean_{wind}_{tf}': 'spread_mean', f'spread_std_{wind}_{tf}': 'spread_std'}).drop_nulls()

    trades_df = backtest(tdf, token_1, token_2, dp_1, dp_2,
            thresh_low_in=-in_, thresh_low_out=-out_, thresh_high_in=in_, thresh_high_out=out_,
            long_possible=True, short_possible=True,
            balance=100, order_size=50, qty_method=qty_method, std_1=std_1, std_2=std_2,
            fee_rate=0.001, sl_std=sl_std, sl_dist=1.0, sl_method='leave',
            sl_seconds = 60, open_method=open_method, close_method=close_method, 
            leverage=leverage, dist_in=dist_in, dist_out=dist_out, force_close=force_close,
            verbose=0)

    if trades_df.height > 1:
        metrics = analyze_strategy(trades_df, start_date=valid_time, end_date=end_date, initial_balance=100.0)
        return tdf, trades_df, metrics
    else:
        return tdf, pl.DataFrame(), dict()

In [3]:
def get_coins_info(token_1, token_2):
    dp_1 = float(coin_information['bybit_linear'][token_1 + '_USDT']['qty_step'])
    ps_1 = int(coin_information['bybit_linear'][token_1 + '_USDT']['price_scale'])
    dp_2 = float(coin_information['bybit_linear'][token_2 + '_USDT']['qty_step'])
    ps_2 = int(coin_information['bybit_linear'][token_2 + '_USDT']['price_scale'])

    return dp_1, dp_2, ps_1, ps_2

In [4]:
pairs_file = './data/token_pairs.txt'

all_pairs = pl.read_parquet('./data/pair_selection/all_pairs.parquet')

with open("./data/coin_information.pkl", "rb") as f:
    coin_information = pickle.load(f)

token_pairs = []
with open(pairs_file, 'r') as file:
    for line in file:
        a, b = line.strip().split()
        token_pairs.append((a, b))
len(token_pairs)

78

In [ ]:
# Параметры
config = load_config('./bot/config/config.yaml')
valid_time = config['valid_time']
end_time = config['end_time']
force_close = False
qty_method = 'usdt_neutral'

leverage = 2
min_order_size = 42
max_order_size = 50
fee_rate = 0.001
sl_ratio = 0.1
sl_std = 5.0
close_method = 'direct'

tf = '1h'
winds = (64, 72, 96)
spr_method = 'lr'
open_methods = ('direct', 'reverse_static', 'reverse_dynamic')
in_min = 1.6
in_max = 2.75
out_min = 0.0
out_max = 0.5
dist_min = 0.05
dist_max = 1.0

metrics_arr = []

In [ ]:
n_iters = 5_000

for _ in tqdm(range(n_iters)):
    token_1, token_2 = random.choice(token_pairs)
    pair_stats = all_pairs.filter((pl.col('coin1') == token_1) & (pl.col('coin2') == token_2))

    if pair_stats.height > 0:
        std_1 = pair_stats['std_1'][0]
        std_2 = pair_stats['std_2'][0]
    else:
        print(token_1, token_2, 'No Stats Data!')
        continue

    file_dist = f'./data/pair_backtest/{token_1}_{token_2}_1h_dist.parquet'
    file_lr = f'./data/pair_backtest/{token_1}_{token_2}_1h_lr.parquet'

    try:
        df = pl.read_parquet(file_lr, low_memory=True, rechunk=True, use_pyarrow=True)
        end_date = df['time'][-1]
    except FileNotFoundError:
        print(f'FileNotFoundError!: {filepath}')
        continue

    in_ = round(random.uniform(in_min, in_max), 2)
    out_ = round(random.uniform(out_min, out_max), 2)
    wind = random.choice(winds)
    open_method = random.choice(open_methods)

    dp_1, dp_2, ps_1, ps_2 = get_coins_info(token_1, token_2)

    if open_method == 'direct':
        dist_in = 0.0
        dist_out = 0.0
    elif open_method in ('reverse_static', 'reverse_dynamic'):
        dist_in = round(random.uniform(dist_min, dist_max), 2)
        dist_out = round(random.uniform(dist_min, dist_max), 2)

    tdf, trades_df, metrics = run_test(df, wind, in_, out_, open_method, dist_in, dist_out, force_close)

    tick_df, agg_df = load_data(token_1, token_2, valid_time, end_time, tf, max(winds), db_manager)
    tick_df = tick_df.drop_nulls()
    

    tick_df = tick_df.with_columns((pl.col(token_1).log() - pl.col(token_2).log()).alias('log_spread'))
    agg_df = agg_df.with_columns((pl.col(token_1).log() - pl.col(token_2).log()).alias('log_spread'))
    
    start_ts = int(datetime.timestamp(valid_time))
    lr_df = create_zscore_df(token_1, token_2, tick_df, agg_df, tf, np.array([wind]), start_ts, 
                                 median_length=6, spr_method='lr')

    break
    for trade in trades_df.to_dicts():
        open_time = trade['open_time']
        side_1 = 'long' if trade['pos_side'] == 1 else 'short'
        side_2 = 'long' if trade['pos_side'] == 2 else 'short'
    
        lr_hist = lr_df.filter(pl.col('time') < open_time).tail(wind * 12 * 60)
        lr_spread = lr_hist[f'spread_{wind}_1h'].to_numpy()
    
        params = {'tf': '1h', 'wind': wind, 'side_1': side_1, 'side_2': side_2}
        try:
            stats = get_open_time_stats(token_1, token_2, open_time, tick_df, agg_df, lr_spread, params, coin_information)
        except IndexError:
            continue
    
        trade.pop('open_time')
        trade.pop('close_time')
        pars = {'token_1': token_1, 'token_2': token_2, 'spr_method': spr_method, 'wind': wind, 
                'in_': in_, 'out_': out_, 'open_method': open_method, 'dist_in': dist_in, 'dist_out': dist_out}
        
        log = pars | trade | stats
        json_log = json.dumps(log, default=float, ensure_ascii=False)
        with open('./logs/backtest_trades.jsonl', 'a', encoding='utf-8') as f:
            f.write(json_log + '\n')
    
        metrics_arr.append(log)

In [ ]:
len(metrics_arr)

In [ ]:
pl.DataFrame(metrics_arr).sample(5)

#### ML

In [5]:
trades = pl.read_ndjson('./logs/backtest_trades.jsonl')
trades = trades.to_dummies(columns=["open_method", "spr_method"]).drop('spr_method_lr', 'open_method_reverse_static'
            ).rename({'spr_method_dist': 'spr_method', 'open_method_direct': 'om_dir', 'open_method_reverse_dynamic': 'om_rd'})

In [8]:
trades.select('token_1', 'token_2').unique()

token_1,token_2
str,str
"""ARKM""","""SEI"""
"""PNUT""","""SEI"""
"""HBAR""","""SEI"""
"""ATOM""","""DOT"""
"""CRV""","""LDO"""
…,…
"""ARKM""","""DRIFT"""
"""RUNE""","""TIA"""
"""ALGO""","""GLM"""


In [ ]:
# Возьмём только линейную регрессию и open_method == reverse_static
trades = trades.filter(
    (pl.col('spr_method') == 0) # LinReg
    & (pl.col('om_dir') == 0) & (pl.col('om_rd') == 0)
)


In [ ]:
print('Размер датасета:', trades.height)
print('Распределение профита на всём датасете.')
print(f'min: {trades['total_profit'].min():.1f}$; max: {trades['total_profit'].max():.1f}$', end=', ')
print(f'mean: {trades['total_profit'].mean():.1f}$, median: {trades['total_profit'].median():.1f}$', end=', ')
print(f'SUM: {trades['total_profit'].sum():.1f}$; std: {trades['total_profit'].std():.2f}$')

In [ ]:
# Распределение профита на всём датасете.
# min: -25.3$; max: 19.3$, mean: -0.4$, median: 0.4$, sum: -2379.5$; std: 3.64$

In [ ]:
# Создаём новые фичи
trades = trades.with_columns(
    (pl.col('open_price_1') / pl.col('open_price_2') - 1).alias('price_ratio'),
    (pl.col('btc_corr_1') / pl.col('btc_corr_2') - 1).alias('btc_corr_ratio'),
    (pl.col('eth_corr_1') / pl.col('eth_corr_2') - 1).alias('eth_corr_ratio'),
    (pl.col('sol_corr_1') / pl.col('sol_corr_2') - 1).alias('sol_corr_ratio'),
    (pl.col('beta_1_180d') / pl.col('beta_2_180d') - 1).alias('beta_180d_ratio'),
    (pl.col('std_1_180d') / pl.col('std_2_180d') - 1).alias('std_180d_ratio'),
    (pl.col('std_1_wind') / pl.col('std_2_wind') - 1).alias('std_wind_ratio'),
    (pl.col('std_1_12h') / pl.col('std_2_12h') - 1).alias('std_12h_ratio'),
).with_columns(
    (abs(pl.col('price_ratio')) < 0.25).alias('price_ratio_0.25'),
    (abs(pl.col('price_ratio')) < 0.5).alias('price_ratio_0.5'),
    (abs(pl.col('price_ratio')) < 0.75).alias('price_ratio_0.75'),
    (abs(pl.col('price_ratio')) < 1.0).alias('price_ratio_1.0'),
    (abs(pl.col('price_ratio')) < 1.25).alias('price_ratio_1.25'),
    (abs(pl.col('price_ratio')) < 1.5).alias('price_ratio_1.5'),
    (abs(pl.col('price_ratio')) < 2.0).alias('price_ratio_2.0'),
    (abs(pl.col('price_ratio')) < 2.5).alias('price_ratio_2.5'),
    
    (abs(pl.col('btc_corr_ratio')) < 0.05).alias('btc_corr_ratio_0.05'),
    (abs(pl.col('btc_corr_ratio')) < 0.2).alias('btc_corr_ratio_0.20'),
    (abs(pl.col('eth_corr_ratio')) < 0.05).alias('eth_corr_ratio_0.05'),
    (abs(pl.col('eth_corr_ratio')) < 0.2).alias('eth_corr_ratio_0.20'),
    (abs(pl.col('sol_corr_ratio')) < 0.05).alias('sol_corr_ratio_0.05'),
    (abs(pl.col('sol_corr_ratio')) < 0.2).alias('sol_corr_ratio_0.20'),

    (abs(pl.col('beta_180d_ratio')) < 0.5).alias('beta_180d_ratio_0.5'),
    (abs(pl.col('beta_180d_ratio')) < 1.0).alias('beta_180d_ratio_1.0'),
    (abs(pl.col('beta_180d_ratio')) < 1.5).alias('beta_180d_ratio_1.5'),

    (abs(pl.col('std_180d_ratio')) < 0.25).alias('std_180d_ratio_0.25'),
    (abs(pl.col('std_180d_ratio')) < 0.5).alias('std_180d_ratio_0.5'),
    (abs(pl.col('std_180d_ratio')) < 1.0).alias('std_180d_ratio_1.0'),
    (abs(pl.col('std_wind_ratio')) < 0.25).alias('std_wind_ratio_0.25'),
    (abs(pl.col('std_wind_ratio')) < 0.5).alias('std_wind_ratio_0.5'),
    (abs(pl.col('std_wind_ratio')) < 1.0).alias('std_wind_ratio_1.0'),
    (abs(pl.col('std_12h_ratio')) < 0.25).alias('std_12h_ratio_0.25'),
    (abs(pl.col('std_12h_ratio')) < 0.5).alias('std_12h_ratio_0.5'),
    (abs(pl.col('std_12h_ratio')) < 1.0).alias('std_12h_ratio_1.0'),

    (pl.col('spread_rsi_5m') < 20).alias('spread_rsi_5m_20'),
    (pl.col('spread_rsi_5m') < 25).alias('spread_rsi_5m_25'),
    (pl.col('spread_rsi_5m') < 30).alias('spread_rsi_5m_30'),
    (pl.col('spread_rsi_5m') < 40).alias('spread_rsi_5m_40'),
    (pl.col('spread_rsi_1h') < 20).alias('spread_rsi_1h_20'),
    (pl.col('spread_rsi_1h') < 25).alias('spread_rsi_1h_25'),
    (pl.col('spread_rsi_1h') < 30).alias('spread_rsi_1h_30'),
    (pl.col('spread_rsi_1h') < 40).alias('spread_rsi_1h_40'),
)

In [ ]:
unhelpful_cols = ['token_1', 'token_2', 'fees', 'qty_1', 'qty_2', 'profit_1', 'profit_2', 'open_price_1', 'open_price_2',
                 'open_ts', 'close_ts', 'reason', 'close_price_1', 'close_price_2', 'pos_side', 'btc_corr_1', 'btc_corr_2',
                 'eth_corr_1', 'eth_corr_2', 'sol_corr_1', 'sol_corr_2', 'std_1_wind', 'std_2_wind', 'std_1_180d', 'std_2_180d',
                 'std_1_12h', 'std_2_12h', 'total_profit']

In [ ]:
feature_cols = ['pos_side', 'in_', 'out_', 'dist_in', 'dist_out', 'wind', 'price_ratio', 'price_ratio_0.25', 'price_ratio_0.5',
               'price_ratio_0.75', 'price_ratio_1.0', 'price_ratio_1.25', 'price_ratio_1.5', 'price_ratio_2.0', 'price_ratio_2.5',
               'pv_1', 'pv_2', 'coint_180d', 
               'profit_sensitivity_12h_pos', 'profit_sensitivity_12h_neg', 'profit_sensitivity_12h_mean', 
               'profit_sensitivity_wind_pos', 'profit_sensitivity_wind_neg', 'profit_sensitivity_wind_mean',
               'btc_corr_ratio', 'eth_corr_ratio', 'sol_corr_ratio',
               'btc_corr_ratio_0.05', 'btc_corr_ratio_0.20', 'eth_corr_ratio_0.05', 'eth_corr_ratio_0.20',
               'sol_corr_ratio_0.05', 'sol_corr_ratio_0.20', 'half_life_log_spread', 'half_life_lr_spread',
               'johansen_beta_180d', 'beta_1_180d', 'beta_2_180d', 'beta_180d_ratio', 'beta_180d_ratio_0.5',
               'beta_180d_ratio_1.0', 'beta_180d_ratio_1.5', 'hurst_wind', 'hurst_12h', 'trend_12h', 'trend_wind',
               'std_180d_ratio', 'std_180d_ratio_0.25', 'std_180d_ratio_0.5', 'std_180d_ratio_1.0',
               'std_wind_ratio', 'std_wind_ratio_0.25', 'std_wind_ratio_0.5', 'std_wind_ratio_1.0', 
               'std_12h_ratio', 'std_12h_ratio_0.25', 'std_12h_ratio_0.5', 'std_12h_ratio_1.0', 
               'mean_diff', 'mean_12h', 'mean_wind', 'spread_rsi_5m', 'spread_rsi_1h',
               'spread_rsi_5m_20', 'spread_rsi_5m_25', 'spread_rsi_5m_30', 'spread_rsi_5m_40',
               'spread_rsi_1h_20', 'spread_rsi_1h_25', 'spread_rsi_1h_30', 'spread_rsi_1h_40',
               'rsi_t1_5m', 'rsi_t2_5m', 'rsi_t1_1h', 'rsi_t2_1h', 'tls_beta_180d', 'tls_beta_wind', 'tls_beta_12h',
               ]
trades.select(feature_cols).sample(3)

In [ ]:
# trades.drop(feature_cols + unhelpful_cols).sample(2)

In [ ]:
# Подготовка данных
profit_threshold = 0.5

X = trades.select(feature_cols).to_numpy()
y_reg = trades.select('total_profit').to_numpy().ravel()
y_clf = trades.select('reason').to_numpy().ravel()
y_profit_bin = (trades.select('total_profit').to_numpy() > profit_threshold).astype(int).ravel()

# Разделение данных
X_train, X_test, y_reg_train, y_reg_test, y_clf_train, y_clf_test, y_train, y_test = train_test_split(
    X, y_reg, y_clf, y_profit_bin, test_size=0.2, random_state=6
)

In [ ]:
print('Распределение профита на тренировочном и тестовом датасете, а также доля стоп-лоссов')
print(f'train ({len(X_train)} записей). min: {y_reg_train.min():.2f}$; max: {y_reg_train.max():.2f}$', end=', ')
print(f'mean: {y_reg_train.mean():.2f}$, median: {np.median(y_reg_train):.2f}$', end=', ')
print(f'std: {y_reg_train.std():.2f}$; sl ratio: {np.unique_counts(y_clf_train).counts[1] / len(y_clf_train):.2f}')

print(f'test  ({len(X_test)} записей).  min: {y_reg_test.min():.2f}$; max: {y_reg_test.max():.2f}$', end=', ')
print(f'mean: {y_reg_test.mean():.2f}$, median: {np.median(y_reg_test):.2f}$', end=', ')
print(f'std: {y_reg_test.std():.2f}$; sl ratio: {np.unique_counts(y_clf_test).counts[1] / len(y_clf_test):.2f}')

In [ ]:
# rf = RandomForestRegressor(random_state=42)
# rf.fit(X_train, y_reg_train);
# rf_preds = rf.predict(X_test)
# root_mean_squared_error(y_reg_test, rf_preds)

In [ ]:
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_clf_train);
rf_preds = rf_clf.predict(X_test)
rf_clf_pred_proba = rf_clf.predict_proba(X_test)[:, 1]
print(f"ROC-AUC: {roc_auc_score(y_clf_test, rf_clf_pred_proba):.3f}")

In [ ]:
import catboost as cb

In [ ]:
# params = {'learning_rate': [0.01, 0.03, 0.1, 0.3], 'iterations': [250, 500, 1000], 
#           'depth': [6, 8, 10],
#           'l2_leaf_reg': [1, 3, 5, 7]}
# cbr = cb.CatBoostRegressor(random_state=42, verbose=False)
# gcv = GridSearchCV(cbr, params, cv=5, verbose=0)
# gcv.fit(X_train.to_numpy(), y_train.to_numpy());

In [ ]:
# gcv.best_params_

In [ ]:
cbr_reg = cb.CatBoostRegressor(random_state=42, verbose=False)
cbr_reg.fit(X_train, y_reg_train);
cbr_reg_preds = cbr_reg.predict(X_test)
root_mean_squared_error(y_reg_test, cbr_reg_preds)

In [ ]:
# 3.1434

In [ ]:
# cbr.save_model('./data/catboost_model.json', format='json')

In [ ]:
# Классификация стоп-лоссов
cbr_clf = cb.CatBoostClassifier(random_state=42, verbose=False)
cbr_clf.fit(X_train, y_clf_train);
cbr_clf_preds = cbr_clf.predict(X_test)
cbr_clf_pred_proba = cbr_clf.predict_proba(X_test)[:, 1]
print(f"ROC-AUC: {roc_auc_score(y_clf_test, cbr_clf_pred_proba):.3f}")

In [ ]:
# ROC-AUC: 0.878

In [ ]:
# Классификация профита > 0.5
cbr_pr_clf = cb.CatBoostClassifier(random_state=42, verbose=False)
cbr_pr_clf.fit(X_train, y_train);
cbr_pr_clf_pred_proba = cbr_pr_clf.predict_proba(X_test)[:, 1]
print(f"ROC-AUC: {roc_auc_score(y_test, cbr_pr_clf_pred_proba):.3f}")

In [ ]:
# ROC-AUC: 0.776

In [ ]:
feature_importance = pl.DataFrame({
    'feature': feature_cols,
    'cb_reg': np.abs(cbr_reg.feature_importances_),
    'cb_clf': np.abs(cbr_clf.feature_importances_),
})
feature_importance.with_columns(
    pl.sum_horizontal('cb_reg', 'cb_clf').alias('sum')
).sort(by='sum', descending=True)

In [ ]:
test_df = pl.DataFrame(X_test, schema=feature_cols, orient='row').with_columns(
        pl.Series('total_profit', y_reg_test),
        pl.Series('reason', y_clf_test),
        pl.Series('reason_pred', cbr_clf_pred_proba),
        pl.Series('profit_pred', cbr_reg_preds),
        pl.Series('good_profit_pred', cbr_pr_clf_pred_proba),
    
    )

In [ ]:
test_df.select('in_', 'out_', 'reason', 'total_profit', 'reason_pred', 'profit_pred', 'good_profit_pred').sample(5)

In [ ]:
# -37.77193336900045
test_df['total_profit'].sum()

In [ ]:
# 85.77454338199976
test_df.filter(pl.col('reason_pred') < 0.05)['total_profit'].sum()

In [ ]:
# 194.9962300989995
test_df.filter(pl.col('profit_pred') > 0.0)['total_profit'].sum()

In [ ]:
# 193.6236992739997
test_df.filter(pl.col('good_profit_pred') > 0.5)['total_profit'].sum()

In [ ]:
# -14.119116149999996
test_df.filter(pl.col('good_profit_pred') > 0.5)['total_profit'].min()

In [ ]:
trading_history = db_manager.get_table('trading_history', df_type='polars').sort(by='close_time')

In [ ]:
for trade in trading_history.iter_rows(named=True):
    token_1 = trade['token_1']
    token_2 = trade['token_2']
    side_1 = trade['side_1']
    side_2 = trade['side_2']
    profit = trade['profit']
    open_time = trade['open_time']
    end_time = trade['close_time']
    start_ts = int(datetime.timestamp(open_time))
    end_ts = int(datetime.timestamp(end_time))

    tf = '1h'
    wind = 96

    tick_df, agg_df = load_data(token_1, token_2, open_time, end_time, tf, wind, db_manager)
    tick_df = tick_df.with_columns((pl.col(token_1).log() - pl.col(token_2).log()).alias('log_spread'))
    agg_df = agg_df.with_columns((pl.col(token_1).log() - pl.col(token_2).log()).alias('log_spread'))

    dist_df = create_zscore_df(token_1, token_2, tick_df, agg_df, tf, np.array([wind]), start_ts, 
                             median_length=6, spr_method='dist')
    lr_df = create_zscore_df(token_1, token_2, tick_df, agg_df, tf, np.array([wind]), start_ts, 
                             median_length=6, spr_method='lr')
    tls_df = create_zscore_df(token_1, token_2, tick_df, agg_df, tf, np.array([wind]), start_ts, 
                             median_length=6, spr_method='tls')
    
    lr_spread = lr_df[f'spread_{wind}_{tf}'].to_numpy()

    
    
    dist_zscore = dist_df.filter((pl.col('time') - open_time) < timedelta(seconds=20)).head(1).select(f'z_score_{wind}_1h').item()
    lr_zscore = lr_df.filter((pl.col('time') - open_time) < timedelta(seconds=20)).head(1).select(f'z_score_{wind}_1h').item()

    params = {'tf': tf, 'wind': wind, 'side_1': side_1, 'side_2': side_2}
    open_time_stats = get_open_time_stats(token_1, token_2, open_time, tick_df, agg_df, lr_spread, params, coin_information)
    
    break

In [ ]:
open_time_stats

In [ ]:
trade